In [1]:
!pip install mip

In [2]:
from mip import Model, maximize,CONTINUOUS, BINARY, xsum
from collections import deque
from queue import PriorityQueue

##Descrição geral

Neste trabalho, vocês deverão implementar, em qualquer linguagem de programação, o algoritmo branch-and-bound para problemas de programação linear inteira binária. A função objetivo será de maximização e todas as restrições serão de "menor ou igual", com exceção daquelas que definem o domínio das variáveis. Não se espera que o(a) discente implemente um algoritmo para resolver os subproblemas (por exemplo, o Simplex), mas que seja usado um pacote de programação linear para resolvê-los (por exemplo, python-mip).

###Regra de ramificação

A regra de ramificação diz como criar os nós filhos a partir de um nó cuja solução da relaxação linear é fracionária. O código de vocês deve ramificar em torno da variável xj cujo valor seja fracionário e mais próximo de 0,5. Em um filho, adiciona-se a restrição xj = 1; no outro, a restrição xj = 0.

###Estratégia de ramificação

A estratégia de ramificação determina como a árvore é explorada, ou seja, qual será o próximo nó a ser resolvido.


## Passo a passo

1. Formular o Problema. Python-mip
2. Criar Nó raiz (representa o problema original)
3. Resolver o Nó raiz (com relaxação)
4. Verificar a Solução. (Se a solução inteira é viável. If not, ramificar)
5. Ramificar (Branching)
    - Identificar a variável mais próxima de 0.5.
    No caso $x_j$.
    - Criar 2 nós filhos.
    x_left =0 , x_rigth = 1
6. Resolver os nós filhos.
7. Poda.
8. Atuaizar UB e LB
9. Continuar a busca
9. Critérios de parada (infeasible ou no solution)


In [28]:
class Problem:
    def __init__(self, objective_coeffs, constraints_coeffs, constraints_rhs):
        self.objective_coeffs = objective_coeffs #coef das funcoes objetivo. ex Max 5x2+6x1. a lista seria [
        self.constraints_coeffs = constraints_coeffs #coefs das restricçoes do problema
        self.constraints_rhs = constraints_rhs # lados direitos das restricoes

        print(objective_coeffs)
        print(constraints_coeffs)
        print(constraints_rhs)

In [41]:
class Node:
    print("entrou na classe no")
    node_count = 0

    def __init__ (self, problem, solution=None, objective_value=None, is_integer_solution=None, parent=None, parent_side=None):
        print("inicializou o no")

        self.node_id = Node.node_count
        Node.node_count += 1 #incrementa o id do node
        self.problem = problem
        self.solution = solution # solução parcial (valor das variáveis)
        self.objective_value = objective_value #(valor da função objetivo)
        self.is_integer_solution = is_integer_solution
        self.parent = parent
        self.parent_side = parent_side # Indica se o nó é filho esquerdo ou direito do pai
        self.left_child = None
        self.right_child = None

    def __str__(self):
        if self.parent:
            parent_id =self.parent.node_id
            return f"Nó {self.node_id} ({self.parent_side} child of {parent_id}), solução = {self.solution}, valor da função objetivo = {self.objective_value}, solução inteira = {self.is_integer_solution}"
        else:
            return f"Nó {self.node_id} (root node)"

    #adicionei pq não tinha como comparar node < / > / == node
      #usei o objetivo de comparação
    def __lt__(self, other):
      return self.objective_value < other.objective_value

    def solve_subproblem(self, branching_var):
      print("entrou no solve_subproblem")
      model = Model()

      print("branching_var: ", branching_var)

      #variaveis tipo x_1 x_2
      variables = [model.add_var(var_type = CONTINUOUS, name=f'x_{i}') for i in range(len(self.problem.objective_coeffs))]
      print("variaveis: ", variables)

      #funcao objetivo
      model.objective = maximize(xsum(self.problem.objective_coeffs[i] * variables[i] for i in range(len(variables))))

      #Restrições
      for constraint_coeffs, rhs in zip(self.problem.constraints_coeffs, self.problem.constraints_rhs):
          model += xsum(constraint_coeffs[i] * variables[i] for i in range(len(variables))) <= rhs

      #Adiciona restrição de ramificação xj = 0 ou xj = 1
      if self.parent_side == 'left':
          model += branching_var == 0
      elif self.parent_side == 'right':
          model += branching_var == 1

      #Resolve o modelo
      model.optimize()

      #está vazia?
      if variables:
        #Verifica se a solução é inteira
        is_integer_solution = all(var.x == int(var.x) for var in variables)

      else:
        print("Variáveis vazias")
        return

      #Atualização de Nó
      self.solution = {var.name: var.x for var in variables}
      self.objective_value = model.objective_value
      self.is_integer_solution = is_integer_solution

      if None in variables:
        print("Algumas variáveis não foram definidas durante a resolução do subproblema.")
        return None

entrou na classe no


In [39]:
def identify_branching_variable(node):
    print("entrou na identify_branching_variable")
    print("no", node)

    variables = node.solution
    print("variaveis na identify", variables)

    if not variables:
      print("Não há solução definida")
      return None

    #pega os coeficientes da função objetivo do problema
    objective_coeffs = node.problem.objective_coeffs
    #verificando se da certo
    print("Objective Coeffs:", objective_coeffs)

    #variáveis com soluções definidas
    valid_variables = [var_name for var_name, value in variables.items() if value is not None and var_name.startswith('x_')]

    if not valid_variables:
      print("Nenhuma variável de decisão com solução definida.")
      return None

    #variável mais próxima de 0.5
    var_to_be_ramified = min(valid_variables, key=lambda var_name: abs(variables[var_name] - 0.5))
    return var_to_be_ramified

In [31]:
def create_children_nodes(node):
    print("entrou na create_children_nodes")
    problem = node.problem
    branching_var = identify_branching_variable(node)

    if not branching_var:
      print("Variável de ramificação não encontrada.")
      return

    node.solve_subproblem(problem, branching_var)

    #filho esquerdo
    if node.left_child is not None:
      print("Nó esquerdo vazio")
    else:
      left_child = Node(problem, solution=dict(node.solution), parent=node, parent_side="left")
      left_child.solution[branching_var] = 0  #variável de ramo = 0
      node.left_child = left_child


    #filho direito
    if node.right_child is not None:
      print("Nó direito vazio")
    else:
      right_child = Node(problem, solution=dict(node.solution), parent=node, parent_side="right")
      right_child.solution[branching_var] = 1  #variável de ramo = 1
      node.right_child = right_child

In [35]:
#criterio de parada forçada = 10

def branch_and_bound(problem, max_no_explorados=10):
  print("entrou na branch_and_bound")
  #inicializa a fila com a raiz
  raiz = Node(problem)
  no_prioritario = PriorityQueue()
  no_prioritario.put((raiz.objective_value, raiz))
  solucao_otima = None #vazia enquanto não preenchermos
  nos_explorados = 0

  #enquanto o no em prioridade existir
  while not no_prioritario.empty() and nos_explorados < max_no_explorados:
    #pula o no atualk e pega o prox
    _, no = no_prioritario.get()

    #se for solução inteira    e   a funçao objt é melhor que atual?
    if no.is_integer_solution and (solucao_otima is None or no.objective_value > solucao_otima.objective_value):
      solucao_otima = no

    #pode ser melhor? (essa é a poda, pois impede a exploração dos subnos)
    if solucao_otima is not None and no.objective_value <= solucao_otima.objective_value:
      #segue para o proximo
      continue

    branching_var = identify_branching_variable(no)
    print("branching_var", branching_var)
    if branching_var:
        no.solve_subproblem(branching_var)

    if no.left_child:
      if no.left_child.objective_value is not None:  # Verifica se o valor de objetivo do nó filho esquerdo não é None
        no_prioritario.put((no.left_child.objective_value, no.left_child))
    if no.right_child:
     if no.right_child.objective_value is not None:  # Verifica se o valor de objetivo do nó filho direito não é None
        no_prioritario.put((no.right_child.objective_value, no.right_child))

    nos_explorados += 1

  return solucao_otima

In [42]:
#teste
objective_coeffs = [10, 8, 1, 3, 3, 8, 30]  #coeficientes da função objt
constraints_coeffs = [
    [10, 8, 9, 9, 7, 6, 10, 20],  #coeficientes
    [6, 6, 3, 6, 3, 7, 2, 80],
    [7, 10, 7, 8, 7, 8, 7, 100],
    [9, 8, 1, 1, 8, 10, 2, 90],
    [1, 5, 3, 10, 2, 4, 9, 70],
    [9, 6, 1, 4, 7, 5, 10, 60],
    [5, 7, 4, 4, 3, 4, 10, 40]
]
constraints_rhs = [8, 9, 7, 8, 7, 8, 7]  #lados direitos das restrições

problema = Problem(objective_coeffs, constraints_coeffs, constraints_rhs) #verificado e funcionando

solucao_otima = branch_and_bound(problema)

if solucao_otima:
    print("Solução ótima encontrada:")
    print(solucao_otima)
else:
    print("Não foi encontrada uma solução ótima.")

print(problema)
print(solucao_otima)

[10, 8, 1, 3, 3, 8, 30]
[[10, 8, 9, 9, 7, 6, 10, 20], [6, 6, 3, 6, 3, 7, 2, 80], [7, 10, 7, 8, 7, 8, 7, 100], [9, 8, 1, 1, 8, 10, 2, 90], [1, 5, 3, 10, 2, 4, 9, 70], [9, 6, 1, 4, 7, 5, 10, 60], [5, 7, 4, 4, 3, 4, 10, 40]]
[8, 9, 7, 8, 7, 8, 7]
entrou na branch_and_bound
inicializou o no
entrou na identify_branching_variable
no Nó 0 (root node)
variaveis na identify None
Não há solução definida
branching_var None
Não foi encontrada uma solução ótima.
None


In [44]:
#@title o que tem que consertar
'''
o solution não acessa nada; o que ele deveria ser?

classe: identify_branching_variable(node):
    variables = node.solution

o erro é nessa linha, pq acaba que o variables feica None

tem que consertar isso
'''

'\no solution não acessa nada; o que ele deveria ser?\n\nclasse: identify_branching_variable(node):\n    variables = node.solution\n\no erro é nessa linha, pq acaba que o variables feica None\n\ntem que consertar isso\n'

In [34]:
#@title ignorado por ora
def read_data(file_name):
    pass